# RNNs -- VRNN, LSTM, GRU, SimpleRNN

Using TensorFlow Keras RNN layers (e.g. LSTM, GRU). The main purpose is to implement VRNN and compare with just LSTM, GRU, etc.

In [1]:
import os
import re
import pathlib
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.python.keras import activations
from tensorflow.python.keras import backend as K

Load a time-series dataset. I'll use the "Plane" dataset from http://www.cs.ucr.edu/~eamonn/time_series_data/

In [2]:
def load_data(fn):
    """
    Load CSV files in UCR time-series data format
    
    Returns:
        data - numpy array with data of shape (num_examples, num_features)
        labels - numpy array with labels of shape: (num_examples, 1)
    """
    df = pd.read_csv(fn, header=None)
    df_data = df.drop(0, axis=1).values.astype(np.float32)
    df_labels = df.loc[:, df.columns == 0].values.astype(np.uint8)
    return df_data, df_labels

train_data, train_labels = load_data("Plane/Plane_TRAIN")
test_data, test_labels = load_data("Plane/Plane_TEST")

# Information about dataset
num_features = 1
time_steps = train_data.shape[1]
num_classes = len(np.unique(train_labels))
data_info = (time_steps, num_features, num_classes)

Implementing a basic RNN cell. Based on TensorFlow [Keras RNN example](https://www.tensorflow.org/api_docs/python/tf/keras/layers/RNN) and [SimpleRNN](https://github.com/tensorflow/tensorflow/blob/r1.10/tensorflow/python/keras/layers/recurrent.py).

In [3]:
class MinimalRNNCell(tf.keras.layers.Layer):
    def __init__(self, units, **kwargs):
        self.units = units
        self.state_size = units
        self.activation = activations.get('tanh')
        super(MinimalRNNCell, self).__init__(**kwargs)

    def build(self, input_shape):
        self.kernel = self.add_weight(shape=(input_shape[-1], self.units),
                                      initializer='glorot_uniform',
                                      name='kernel')
        self.recurrent_kernel = self.add_weight(
            shape=(self.units, self.units),
            initializer='orthogonal',
            name='recurrent_kernel')
        # https://github.com/tensorflow/tensorflow/blob/r1.10/tensorflow/python/keras/engine/base_layer.py
        # says to do this not "self.built = True" like was given in the example
        super(MinimalRNNCell, self).build(input_shape)

    def call(self, inputs, states):
        prev_output = states[0]
        h = K.dot(inputs, self.kernel)
        output = h + K.dot(prev_output, self.recurrent_kernel)
        output = self.activation(output)
        return output, [output]

    def get_config(self):
        """
        Allow loading the init args from the saved model
        https://github.com/keras-team/keras/issues/5401#issuecomment-280100357
        """
        config = {'units': self.units}
        base_config = super(MinimalRNNCell, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

Implementing VRNN. Based on:
 - https://github.com/phreeza/tensorflow-vrnn/blob/master/model_vrnn.py
 - https://github.com/kimkilho/tensorflow-vrnn/blob/master/cell.py
 - https://github.com/kimkilho/tensorflow-vrnn/blob/master/main.py

In [11]:
class VRNNCell(tf.keras.layers.Layer):
    """
    VRNN cell
    
    Usage:
        cell = VRNNCell(h_dim, z_dim)
        net = tf.keras.layers.RNN(cell)(net)
    """
    def __init__(self, x_dim, h_dim, z_dim, batch_size, **kwargs):
        self.batch_size = batch_size
        
        # Dimensions of x input, hidden layers, latent variable (z)
        self.n_x = x_dim
        self.n_h = h_dim
        self.n_z = z_dim
        
        # Dimensions of phi(z)
        self.n_x_1 = x_dim
        self.n_z_1 = z_dim
        
        # Dimensions of encoder, decoder, and prior
        self.n_enc_hidden = z_dim
        self.n_dec_hidden = x_dim
        self.n_prior_hidden = z_dim
        
        # What cell we're going to use internally for the RNN
        self.cell = tf.keras.layers.LSTMCell(self.n_h,
             input_shape=(None, self.n_dec_hidden+self.n_z_1))
        
        super(VRNNCell, self).__init__(**kwargs)
    
    @property
    def state_size(self):
        # Note: first two are the state of the LSTM
        return (self.n_h, self.n_h,
                self.n_z, self.n_z,
                self.n_x, self.n_x,
                self.n_z, self.n_z)
    
    @property
    def output_size(self):
        return self.n_h

    def build(self, input_shape):
        # Input: previous hidden state
        self.prior_h = self.add_weight(
            shape=(self.n_h, self.n_prior_hidden), initializer='glorot_uniform', name='prior_hidden')
        self.prior_mu = self.add_weight(
            shape=(self.n_prior_hidden, self.n_z), initializer='glorot_uniform', name='prior_mu')
        self.prior_sigma = self.add_weight(
            shape=(self.n_prior_hidden, self.n_z), initializer='glorot_uniform', name='prior_sigma')
        
        self.prior_h_b = self.add_weight(
            shape=(self.n_prior_hidden,), initializer='constant', name='prior_hidden_b')
        self.prior_mu_b = self.add_weight(
            shape=(self.n_z,), initializer='constant', name='prior_mu_b')
        self.prior_sigma_b = self.add_weight(
            shape=(self.n_z,), initializer='constant', name='prior_sigma_b')
        
        # Input: x
        self.x_1 = self.add_weight(
            shape=(self.n_x, self.n_x_1), initializer='glorot_uniform', name='phi_x')
        
        self.x_1_b = self.add_weight(
            shape=(self.n_x_1,), initializer='constant', name='phi_x_b')
        
        # Input: x and previous hidden state
        self.encoder_h = self.add_weight(
            shape=(self.n_x_1+self.n_h, self.n_enc_hidden), initializer='glorot_uniform', name='encoder_hidden')
        self.encoder_mu = self.add_weight(
            shape=(self.n_enc_hidden, self.n_z), initializer='glorot_uniform', name='encoder_mu')
        self.encoder_sigma = self.add_weight(
            shape=(self.n_enc_hidden, self.n_z), initializer='glorot_uniform', name='encoder_sigma')
        
        self.encoder_h_b = self.add_weight(
            shape=(self.n_enc_hidden,), initializer='constant', name='encoder_hidden_b')
        self.encoder_mu_b = self.add_weight(
            shape=(self.n_z,), initializer='constant', name='encoder_mu_b')
        self.encoder_sigma_b = self.add_weight(
            shape=(self.n_z,), initializer='constant', name='encoder_sigma_b')
        
        # Input: z = enc_sigma*eps + enc_mu -- i.e. reparameterization trick
        self.z_1 = self.add_weight(
            shape=(self.n_z, self.n_z_1), initializer='glorot_uniform', name='phi_z')
        
        self.z_1_b = self.add_weight(
            shape=(self.n_z_1,), initializer='constant', name='phi_z_b')
        
        # Input: latent variable (z) and previous hidden state
        self.decoder_h = self.add_weight(
            shape=(self.n_z+self.n_h, self.n_dec_hidden), initializer='glorot_uniform', name='decoder_hidden')
        self.decoder_mu = self.add_weight(
            shape=(self.n_dec_hidden, self.n_x), initializer='glorot_uniform', name='decoder_mu')
        self.decoder_sigma = self.add_weight(
            shape=(self.n_dec_hidden, self.n_x), initializer='glorot_uniform', name='decoder_sigma')
        
        self.decoder_h_b = self.add_weight(
            shape=(self.n_dec_hidden,), initializer='constant', name='decoder_hidden_b')
        self.decoder_mu_b = self.add_weight(
            shape=(self.n_x,), initializer='constant', name='decoder_mu_b')
        self.decoder_sigma_b = self.add_weight(
            shape=(self.n_x,), initializer='constant', name='decoder_sigma_b')
        
        super(VRNNCell, self).build(input_shape)

    def call(self, inputs, states):
        h = states[0]
        c = states[1] # only passed to the LSTM
        
        # Input: previous hidden state (h)
        prior_h = K.relu(K.dot(h, self.prior_h) + self.prior_h_b)
        prior_mu = K.dot(prior_h, self.prior_mu) + self.prior_mu_b
        prior_sigma = K.softplus(K.dot(prior_h, self.prior_sigma) + self.prior_sigma_b) # >= 0
        
        # Input: x
        x_1 = K.relu(K.dot(inputs, self.x_1) + self.x_1_b) # >= 0
        
        # Input: x and previous hidden state
        encoder_input = K.concatenate((x_1, h), 1)
        encoder_h = K.relu(K.dot(encoder_input, self.encoder_h) + self.encoder_h_b)
        encoder_mu = K.dot(encoder_h, self.encoder_mu) + self.encoder_mu_b
        encoder_sigma = K.softplus(K.dot(encoder_h, self.encoder_sigma) + self.encoder_sigma_b)
        
        # Input: z = enc_sigma*eps + enc_mu -- i.e. reparameterization trick
        #eps = K.random_normal((self.batch_size, self.n_z), dtype=tf.float32)
        eps = K.random_normal((self.batch_size, self.n_z), dtype=tf.float32)
        z = encoder_sigma*eps + encoder_mu
        z_1 = K.relu(K.dot(z, self.z_1) + self.z_1_b)
        
        # Input: latent variable (z) and previous hidden state
        decoder_input = K.concatenate((z_1, h), 1)
        decoder_h = K.relu(K.dot(decoder_input, self.decoder_h) + self.decoder_h_b)
        decoder_mu = K.dot(decoder_h, self.decoder_mu) + self.decoder_mu_b
        decoder_sigma = K.softplus(K.dot(decoder_h, self.decoder_sigma) + self.decoder_sigma_b)
        
        # Pass to cell (e.g. LSTM). Note that the LSTM has both "h" and "c" that are combined
        # into the same next state vector. We'll combine them together to pass in and split them
        # back out after the LSTM returns the next state.
        rnn_cell_input = K.concatenate((x_1, z_1), 1)
        output, (h_next, c_next) = self.cell(rnn_cell_input, [h, c])
        
        # VRNN state
        next_state = (
            h_next,
            c_next,
            encoder_mu,
            encoder_sigma,
            decoder_mu,
            decoder_sigma,
            prior_mu,
            prior_sigma,
        )
        
        return output, next_state

    def get_config(self):
        """ Save cell config to the model file """
        config = {
            'batch_size': self.batch_size,
            'x_dim': self.n_x,
            'h_dim': self.n_h,
            'z_dim': self.n_z,
        }
        base_config = super(VRNNCell, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

Train and test.

In [18]:
def get_dataset(features, labels, num_classes, batch_size, evaluation=False, buffer_size=5000):
    """
    Get the dataset object for feeding into the model
    
    If batch_size==None, then one-hot encode but don't batch (evaluation)
    If batch_size!=None, then repeat, shuffle, and batch (training)
    """
    def map_func(x, y):
        """ One-hot encode y, convert to appropriate data types """
        x_out = tf.cast(tf.expand_dims(x,axis=1), tf.float32)
        y_out = tf.one_hot(tf.squeeze(tf.cast(y, tf.uint8)), depth=num_classes)
        return [x_out, y_out]
    
    dataset = tf.data.Dataset.from_tensor_slices((features, labels))
    dataset = dataset.map(map_func)
    
    if evaluation:
        dataset = dataset.batch(batch_size)
    else:
        dataset = dataset.repeat().shuffle(buffer_size).batch(batch_size)
    
    return dataset

def get_model(time_steps, num_features, num_classes, batch_size, layer_type):
    """ Define RNN model """
    if layer_type == 'lstm':
        layer1 = tf.keras.layers.LSTM(128, return_sequences=True)
        layer2 = tf.keras.layers.LSTM(128, return_sequences=False)
    elif layer_type == 'rnn':
        layer1 = tf.keras.layers.SimpleRNN(128, return_sequences=True)
        layer2 = tf.keras.layers.SimpleRNN(128, return_sequences=False)
    elif layer_type == 'custom_rnn':
        layer1 = tf.keras.layers.RNN(MinimalRNNCell(128), return_sequences=True)
        layer2 = tf.keras.layers.RNN(MinimalRNNCell(128), return_sequences=False)
    elif layer_type == 'vrnn':
        input_size = num_features # x input features
        layer1 = tf.keras.layers.RNN(VRNNCell(input_size, 128, 32, batch_size), return_sequences=True)
        input_size = 128 # size of previous layer output
        layer2 = tf.keras.layers.RNN(VRNNCell(input_size, 128, 32, batch_size), return_sequences=False)
    elif layer_type == 'gru':
        layer1 = tf.keras.layers.GRU(128, return_sequences=True)
        layer2 = tf.keras.layers.GRU(128, return_sequences=False)
    
    x = tf.keras.Input((time_steps,1), dtype=tf.float32)
    n = layer1(x)
    n = tf.keras.layers.Dropout(0.5)(n)
    n = layer2(n)
    n = tf.keras.layers.Dropout(0.5)(n)
    n = tf.keras.layers.Dense(num_classes)(n)
    y = tf.keras.layers.Activation('softmax')(n)
    model = tf.keras.Model(x, y)
    
    if layer_type == 'vrnn':
        model.compile(loss=tf.keras.losses.categorical_crossentropy,
                      #optimizer=tf.keras.optimizers.Adam(), # gives gradient None error
                      optimizer=tf.train.AdamOptimizer(),
                      metrics=['accuracy'])
    else:
        model.compile(loss=tf.keras.losses.categorical_crossentropy,
                      optimizer=tf.keras.optimizers.Adam(),
                      metrics=['accuracy'])
    
    return model

def latest_checkpoint(model_file):
    """
    Find latest checkpoint
    https://www.tensorflow.org/tutorials/keras/save_and_restore_models
    """
    model_path = os.path.dirname(model_file)
    checkpoints = pathlib.Path(model_path).glob("*.hdf5")
    checkpoints = sorted(checkpoints, key=lambda cp:cp.stat().st_mtime)
    checkpoints = [cp.with_suffix('.hdf5') for cp in checkpoints]
    
    if len(checkpoints) > 0:
        # Get epoch number from filename
        regex = re.compile(r'\d\d+')
        numbers = [int(x) for x in regex.findall(str(checkpoints[-1]))]
        assert len(numbers) == 1, "Could not determine epoch number from filename since multiple numbers"
        epoch = numbers[0]
        
        return str(checkpoints[-1]), epoch
    
    return None, None

def train(data_info, features, labels,
          batch_size=64,
          num_epochs=10,
          model_file="models/{epoch:04d}.hdf5",
          log_dir="logs",
          layer_type="lstm"):
    
    model_path = os.path.dirname(model_file)
    if not os.path.exists(model_path):
        os.makedirs(model_path)
    latest, epoch = latest_checkpoint(model_file)

    # Data stats
    time_steps, num_features, num_classes = data_info

    # Get dataset / model
    dataset = get_dataset(features, labels, num_classes, batch_size)
    
    # Load previous weights if found, if not we'll start at epoch 0
    if latest is not None:
        # Load the entire saved model
        #model = tf.keras.models.load_model(latest, custom_objects={
        #    'MinimalRNNCell': MinimalRNNCell,
        #    'VRNNCell': VRNNCell,
        #})
        
        # Alternatively, recreate model and load only the weights from the model file
        model = get_model(time_steps, num_features, num_classes, batch_size, layer_type)
        model.load_weights(latest)
    else:
        model = get_model(time_steps, num_features, num_classes, batch_size, layer_type)
        epoch = 0
    
    # Train
    model.fit(dataset, initial_epoch=epoch, epochs=num_epochs, steps_per_epoch=30, callbacks=[
        # save_weights_only doesn't work for LSTM apparently, the just-trained model.get_weights()
        # don't show up in the model-from-saved-file model.get_weights(), though some are loaded
        # like the last dense layer. This is a saving problem definitely since saving the entire
        # model and then loading just the weights works fine.
        tf.keras.callbacks.ModelCheckpoint(model_file, period=1, verbose=0),
        tf.keras.callbacks.TensorBoard(log_dir),
        tf.keras.callbacks.TerminateOnNaN()
    ])
    
    return model

def evaluate(data_info, features, labels, model=None,
             model_file="models/{epoch:04d}.hdf5",
             layer_type="lstm",
             batch_size=64,
             useTensorFlowDataset=True):
    
    latest, epoch = latest_checkpoint(model_file)
    
    # Data stats
    time_steps, num_features, num_classes = data_info
    
    # Get dataset
    if useTensorFlowDataset:
        dataset = get_dataset(features, labels, num_classes, 1, evaluation=True)
    else:
        x = np.expand_dims(features,axis=2).astype(np.float32)
        y = np.eye(num_classes)[np.squeeze(labels).astype(np.uint8) - 1] # one-hot encode
    
    # Load weights from last checkpoint if model is not given
    if model is None:
        assert latest is not None, "No latest checkpoint to use for evaluation"
        print("Loading model from", latest, "at epoch", epoch)
        
        # Load entire model
        #model = tf.keras.models.load_model(latest, custom_objects={
        #    'MinimalRNNCell': MinimalRNNCell,
        #    'VRNNCell': VRNNCell,
        #})
        
        # Alternatively, recreate model and load only the weights from the model file
        model = get_model(time_steps, num_features, num_classes, batch_size, layer_type)
        model.load_weights(latest)
    
    # Evaluate
    if useTensorFlowDataset:
        loss, acc = model.evaluate(dataset, steps=len(labels))
    else:
        loss, acc = model.evaluate(x, y)
    
    return acc

Run training and evaluation.

In [19]:
#for layer_type in ['lstm', 'rnn', 'gru', 'custom_rnn', 'vrnn']:
for layer_type in ['vrnn']:
    print("Training model:", layer_type)
    tf.reset_default_graph()
    K.clear_session()
    model = train(data_info, train_data, train_labels,
                  model_file=layer_type+"-models/{epoch:04d}.hdf5",
                  log_dir=layer_type+"-logs", layer_type=layer_type)

Training model: vrnn
Epoch 1/10
30/30 [==============================] - 12s 407ms/step - loss: 1.5508 - acc: 0.1536
Epoch 2/10
30/30 [==============================] - 12s 415ms/step - loss: 1.4605 - acc: 0.1552
Epoch 3/10
30/30 [==============================] - 15s 487ms/step - loss: 1.4385 - acc: 0.1583
Epoch 4/10
30/30 [==============================] - 13s 442ms/step - loss: 1.4965 - acc: 0.1422
Epoch 5/10
30/30 [==============================] - 13s 435ms/step - loss: 1.4693 - acc: 0.1750
Epoch 6/10
30/30 [==============================] - 13s 440ms/step - loss: 1.4604 - acc: 0.1661
Epoch 7/10
30/30 [==============================] - 13s 435ms/step - loss: 1.4350 - acc: 0.1510
Epoch 8/10
30/30 [==============================] - 14s 455ms/step - loss: 1.4501 - acc: 0.1620
Epoch 9/10
30/30 [==============================] - 15s 498ms/step - loss: 1.4541 - acc: 0.1531
Epoch 10/10
30/30 [==============================] - 15s 502ms/step - loss: 1.4482 - acc: 0.1693


In [20]:
for layer_type in ['lstm', 'rnn', 'gru', 'custom_rnn', 'vrnn']:
    print("Evaluating model:", layer_type)
    print("  Train:", evaluate(data_info, train_data, train_labels,
                               model_file=layer_type+"-models/{epoch:04d}.hdf5",
                               layer_type=layer_type))
    print("  Test:", evaluate(data_info, test_data, test_labels,
                              model_file=layer_type+"-models/{epoch:04d}.hdf5",
                              layer_type=layer_type))

Evaluating model: lstm
Loading model from lstm-models/0010.hdf5 at epoch 10
105/105 [==============================] - 4s 38ms/step
  Train: 0.8095238095238095
Loading model from lstm-models/0010.hdf5 at epoch 10
105/105 [==============================] - 4s 37ms/step
  Test: 0.8380952380952381
Evaluating model: rnn
Loading model from rnn-models/0010.hdf5 at epoch 10
105/105 [==============================] - 1s 10ms/step
  Train: 0.8
Loading model from rnn-models/0010.hdf5 at epoch 10
105/105 [==============================] - 1s 12ms/step
  Test: 0.8857142857142857
Evaluating model: gru
Loading model from gru-models/0010.hdf5 at epoch 10
105/105 [==============================] - 3s 28ms/step
  Train: 0.7238095238095238
Loading model from gru-models/0010.hdf5 at epoch 10
105/105 [==============================] - 3s 26ms/step
  Test: 0.7238095238095238
Evaluating model: custom_rnn
Loading model from custom_rnn-models/0010.hdf5 at epoch 10
105/105 [==============================] - 1s